## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,80.85,73,58,17.20,broken clouds
1,1,Pakxan,LA,18.3778,103.6601,83.52,59,84,3.18,broken clouds
2,2,Sao Filipe,CV,14.8961,-24.4956,74.86,63,3,9.89,clear sky
3,3,Huichapan,MX,20.3833,-99.6500,80.60,12,1,8.05,clear sky
4,4,Khatanga,RU,71.9667,102.5000,19.74,94,100,15.17,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimun Temp: "))
max_temp = float(input("Enter maximum Temp: "))

Enter minimun Temp: 60
Enter maximum Temp: 65


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

prefered_temp = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                 (city_data_df["Max Temp"] <= max_temp)]

prefered_temp.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Albany,US,42.6001,-73.9662,64.99,58,100,2.82,overcast clouds
6,6,Busselton,AU,-33.6500,115.3333,60.01,88,100,8.01,light rain
18,18,Bunia,CD,1.5667,30.2500,63.82,98,100,5.97,light rain
19,19,Williston,US,48.1470,-103.6180,64.40,17,75,12.66,broken clouds
38,38,Hamilton,US,39.1834,-84.5333,60.01,93,91,9.42,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
prefered_temp.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
prefered_temp = prefered_temp.dropna()
prefered_temp.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = prefered_temp[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Albany,US,64.99,overcast clouds,42.6001,-73.9662,
6,Busselton,AU,60.01,light rain,-33.6500,115.3333,
18,Bunia,CD,63.82,light rain,1.5667,30.2500,
19,Williston,US,64.40,broken clouds,48.1470,-103.6180,
38,Hamilton,US,60.01,overcast clouds,39.1834,-84.5333,
55,Vredendal,ZA,63.66,overcast clouds,-31.6683,18.5012,
59,Ribeira Grande,PT,62.08,overcast clouds,38.5167,-28.7000,
68,Rawson,AR,62.71,scattered clouds,-43.3002,-65.1023,
70,Soni,TZ,61.75,overcast clouds,-4.8500,38.3667,
75,Cape Town,ZA,63.00,clear sky,-33.9258,18.4232,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... Skipping.")

# indicate script completion
print("Done!")
        

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Done!


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()  # shows me 0 null values in my df!



City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [33]:
clean_hotel_df.head() # but there is a null value in first row?! 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Albany,US,64.99,overcast clouds,42.6001,-73.9662,
6,Busselton,AU,60.01,light rain,-33.6500,115.3333,Observatory Guest House
18,Bunia,CD,63.82,light rain,1.5667,30.2500,Gold Star International Hotel
19,Williston,US,64.40,broken clouds,48.1470,-103.6180,Hampton Inn & Suites Williston
38,Hamilton,US,60.01,overcast clouds,39.1834,-84.5333,Six Acres Bed & Breakfast


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F and {Current Description}</dd>
</dl> 
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))